In [31]:
#add necessary libraries
import networkx as nx #library supporting networks
import matplotlib.pyplot as plt #plotting
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy.stats as stat
from scipy import optimize
import pysal as ps
from zipfile import ZipFile
from StringIO import StringIO
#make sure plots are embedded into the notebook
%pylab inline 
import statsmodels.formula.api as smf

Populating the interactive namespace from numpy and matplotlib


## 1. Data Loading

In [12]:
# load state gdp data (2014)
gdp = pd.read_csv('bea_gdp_by_state_in_millions.csv', header = 0, names=['fips', 'state', 'gdp'], index_col=['state'])
print gdp.head()
print gdp.columns
print len(gdp)

           state      gdp
fips                     
1000     Alabama   199440
2000      Alaska    57080
4000     Arizona   284156
5000    Arkansas   121395
6000  California  2311616
Index([u'state', u'gdp'], dtype='object')
51


In [11]:
# load state capitals
caps = pd.read_csv('Capitals.csv', header = 0, index_col=['state'])
print caps.head()
print caps.columns
print len(caps)

   abbrev       state      capital   latitude   longitude  population
id                                                                   
1      AL     Alabama   Montgomery  32.380120  -86.300629      205764
2      AK      Alaska       Juneau  58.299740 -134.406794       31275
4      AZ     Arizona      Phoenix  33.448260 -112.075774     1445632
5      AR    Arkansas  Little Rock  34.748655  -92.274494      193524
6      CA  California   Sacramento  38.579065 -121.491014      466488
Index([u'abbrev', u'state', u'capital', u'latitude', u'longitude',
       u'population'],
      dtype='object')
50


In [14]:
# load state population centers
popcenter = pd.read_csv('CenPop2010_Mean_ST.txt', index_col=['STNAME'])
print popcenter.head()
print popcenter.columns

            STATEFP  POPULATION   LATITUDE   LONGITUDE
STNAME                                                
Alabama           1     4779736  33.008097  -86.756826
Alaska            2      710231  61.399882 -148.873973
Arizona           4     6392017  33.368266 -111.864310
Arkansas          5     2915918  35.142580  -92.655243
California        6    37253956  35.463595 -119.325359
Index([u'STATEFP', u'POPULATION', u'LATITUDE', u'LONGITUDE'], dtype='object')


In [20]:
# load state tax rates (all types)
tax = pd.read_excel('Taxes rates by state.xlsx', index_col=['State'])
#print tax.head()
print tax.columns
print len(tax)

Index([             u'State Sales Tax Rate',
           u'Avg, Local Sales Tax Rate (a)',
                 u'Combined Sales Tax Rate',
                u'Maximum Local Sales Rate',
                        u'Poperty Tax rate',
        u'Individual Income tax rate (low)',
       u'Individual Income tax rate (High)',
          u'Corporate Mature Firm Tax Rate',
             u'Corporate New Firm tax Rate'],
      dtype='object')
51


In [39]:
# This is only for the shapes that will be used by PySAL to 
# build the spatial weights matrix
data = gpd.read_file('cb_2014_us_state_5m/cb_2014_us_state_5m.shp')
psGeom = ps.open('cb_2014_us_state_5m/cb_2014_us_state_5m.shp', 'r')

print data.columns

Index([u'AFFGEOID',    u'ALAND',   u'AWATER',    u'GEOID',     u'LSAD',
           u'NAME',  u'STATEFP',  u'STATENS',   u'STUSPS', u'geometry'],
      dtype='object')


## 2. Build spatial weight matrices

### 2.1. Rook 

In [42]:
# We are building the spatial weight matrix and using the 
# state names as IDs of the matrix.

R = ps.buildContiguity(psGeom, criterion='rook', ids=data['NAME'].values.tolist())

Island ids:  [u'Puerto Rico', u'Commonwealth of the Northern Mariana Islands', u'Alaska', u'Hawaii', u'United States Virgin Islands', u'American Samoa', u'Guam']


In [45]:
#for (loc, neighbors) in R:
    #print loc, neighbors

### 2.2. Queen 

In [46]:
# We are building the spatial weight matrix and using the 
# state names as IDs of the matrix. Noted that we
# running a 'queen', shared vertices, neighborhood test.

Q = ps.buildContiguity(psGeom, criterion='queen', ids=data['NAME'].values.tolist())

Island ids:  [u'Puerto Rico', u'Commonwealth of the Northern Mariana Islands', u'Alaska', u'Hawaii', u'United States Virgin Islands', u'American Samoa', u'Guam']


In [47]:
#for (loc, neighbors) in Q:
    #print loc, neighbors

### 2.3. Distance-weighted by state capitals (50 by 50 matrix) 

### 2.4. Distance-weighted by population centers (50 by 50 matrix) 

### 2.5. Gravity model (???) 

## 3. Calculate spatial auto-correlations for tax rates (each type)

In [ ]:
# Morans I (see lab9_sa from NYU Classes)